In [2]:
import pyarrow.feather as feather


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
df = pd.read_csv('StockEtablissement_utf8.csv', nrows = 100000, skipinitialspace=True)
for k in df.columns:
    print(f'{k} {df[k].isna().sum()}')


siren 0
nic 0
siret 0
statutDiffusionEtablissement 0
dateCreationEtablissement 20708
trancheEffectifsEtablissement 22373
anneeEffectifsEtablissement 72580
activitePrincipaleRegistreMetiersEtablissement 98486
dateDernierTraitementEtablissement 16679
etablissementSiege 0
nombrePeriodesEtablissement 0
complementAdresseEtablissement 74323
numeroVoieEtablissement 21215
indiceRepetitionEtablissement 97308
typeVoieEtablissement 8262
libelleVoieEtablissement 3765
codePostalEtablissement 20
libelleCommuneEtablissement 20
libelleCommuneEtrangerEtablissement 99999
distributionSpecialeEtablissement 99572
codeCommuneEtablissement 20
codeCedexEtablissement 99817
libelleCedexEtablissement 99822
codePaysEtrangerEtablissement 99998
libellePaysEtrangerEtablissement 99998
complementAdresse2Etablissement 99992
numeroVoie2Etablissement 99855
indiceRepetition2Etablissement 99990
typeVoie2Etablissement 99815
libelleVoie2Etablissement 99814
codePostal2Etablissement 99814
libelleCommune2Etablissement 99814
lib

In [4]:
df.shape

(10000000, 5)

In [4]:
df.head()

,siret,codePostalEtablissement,libelleCommuneEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement
0,32517500016,98770.0,MANIHI,98727,32.12Z
1,32517500024,84140.0,AVIGNON,84007,47.89Z
2,32517500032,84000.0,AVIGNON,84007,32.12Z
3,32517500040,13420.0,GEMENOS,13042,32.12Z
4,32517500057,13004.0,MARSEILLE 4,13204,32.12Z


In [5]:
for k in df.columns:
    print(k)
    print(df[k].isna().sum())

siret
0
codePostalEtablissement
25536
libelleCommuneEtablissement
25536
codeCommuneEtablissement
25536
activitePrincipaleEtablissement
9191


## Upload du csv sur la base de données MySQL

In [4]:
#Fonction de connexion au serveur DataIA
import sqlite3
import pymysql
import yaml
from sqlalchemy import create_engine
import pandas as pd

def connexion_mysql():
    with open ('connect.yml', 'r') as f:
        conf = yaml.safe_load(f)
    my = conf['MYSQL']
    conf['MYSQL']['user']
    engineMY = create_engine(f"mysql+pymysql://{my['user']}:{my['password']}@{my['host']}:{my['port']}/bdd_menou", echo=False)
    return(engineMY)

In [8]:
#Upload de la base de données par chunk de 100 000 lignes

engine = connexion_mysql()
L = ['siret', 'numeroVoieEtablissement', 'typeVoieEtablissement', 'libelleVoieEtablissement', 'codePostalEtablissement', 'codeCommuneEtablissement', 'dateDebut', 'etatAdministratifEtablissement', 'activitePrincipaleEtablissement']
df = pd.read_csv('StockEtablissement_utf8.csv', chunksize = 100000, usecols= L)
for i, k in enumerate(df) :
    print(f'chunk {i}')
    k.to_sql('Siren', con=engine, if_exists='append', index=False)

C:\Users\kylli\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


chunk 0
chunk 1
chunk 2
chunk 3
chunk 4
chunk 5
chunk 6
chunk 7
chunk 8
chunk 9
chunk 10
chunk 11
chunk 12
chunk 13
chunk 14
chunk 15
chunk 16
chunk 17
chunk 18
chunk 19
chunk 20
chunk 21


KeyboardInterrupt: 

In [7]:
#Creation d'une fonction renvoyant une carte avec un marqueur par entreprise française pour une catégorie donnée
import folium
import pandas as pd

def Category_Mapping(cat):
    engine = connexion_mysql()
    df_cat = pd.read_sql_query(('''SELECT  POSTE.nomCommune,
                                POSTE.latitude,
                                POSTE.longitude,
                                SIRENE.siret,
                                SIRENE.typeVoieEtablissement,
                                SIRENE.libelleVoieEtablissement,
                                NAF.description
                                FROM SIRENE
                                JOIN POSTE ON SIRENE.codeCommuneEtablissement = POSTE.codeCommune
                                JOIN NAF ON SIRENE.activitePrincipaleEtablissement = NAF.naf
                                WHERE (NAF.description = %s);'''),
                                engine,
                                params=[cat])
    
    CentreFrance = [46.3622, 1.5231]

    map = folium.Map(location = CentreFrance,
                     zoom_start = 6,
                     tiles="Stamen Terrain")
    for idx in df_cat.index:
        tooltip = ""
        folium.Marker([df_cat['longitude'][idx], df_cat['latitude'][idx]], popup="<i></i>", tooltip=tooltip).add_to(map)
    return(map)

In [8]:
Rizi = Category_Mapping(cat = 'Culture du riz')

In [9]:
type(Rizi)

folium.folium.Map